# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [3]:
import numpy as np
import pandas as pd

In [23]:
recommended_list = [156, 1134, 143, 27, 1543, 3345, 533, 11, 43, 991] #id товаров
bought_list = [521, 32, 143, 991]
prices_recommended = [400, 60, 40, 90, 240, 60, 140, 350, 280, 30]
prices_bought = [50, 75, 40, 30]

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [9]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [ ]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)

In [12]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [14]:
def recall_at_k(recommended_list, bought_list, k=5):    
    return recall(recommended_list[:k], bought_list)

In [26]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommend_list, bought_list)
    
    money_recall = np.dot(flags, prices_recommended).sum() / prices_bought.sum()
    
    return money_recall

In [10]:
recall(recommended_list, bought_list)

0.5

In [15]:
# Для того, чтобы проверить что правильно считает изменил немного данные
recall_at_k(recommended_list, bought_list)

0.25

In [27]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, 5)

0.20512820512820512

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [34]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(recommended_list, bought_list)

    # если среди рекомендаций не было ни одной покупки
    if (not np.any(flags)):
        return np.inf

    rank = np.argmax(flags) + 1
    
    return 1/rank

In [35]:
mrr_at_k(recommended_list, bought_list, k=5)

0.3333333333333333

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [38]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
     
    flags = np.isin(recommended_list, bought_list)
    
    i = np.arange(1, k+1)
    discount = np.where(i<=2, i, np.log2(i))
    
    dcg = (flags / discount).sum() / k
    idcg = (np.ones(discount.shape) / discount).sum() / k
    
    return dcg/idcg

In [39]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.20607801570427584